<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-21 11:49:59
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.39 C
-------------------
Today PnL: 13.52 K (0.1%)
Current PnL: -16.81 L (-11.43%)
CY Booked + Current PnL: -8.08 L (-5.49%)
-------------------
Total profit:  3.86 L
Total loss:  -20.67 L
-------------------
Total Booked + Current PnL: 18.90 L (15.75%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.3%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 75.92 L (54.81%)
Deployed:  1.20 C
Current:  1.39 C
CAGR/XIRR %: 9.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.04,113136.0,8824.0,7829.0,0.89,8.46,6.92,15.96,9.0,1.13,0.84,26.79,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.92,220308.0,18828.0,8394.0,0.29,9.35,3.81,13.51,60.0,2.24,1.63,23.12,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.30,173866.0,17689.0,8398.0,0.25,11.33,4.83,16.71,22.0,2.11,1.29,30.74,X40N,BTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-5.92,H-LC,6.95,295904.0,25557.0,9084.0,-1.04,9.45,3.07,12.81,24.0,2.81,2.19,23.75,XY25,NTT,FMCG
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,161124.0,34924.0,11246.0,0.80,27.67,6.98,36.59,108.0,3.11,1.19,43.65,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.37,179115.0,-693.0,113111.0,0.11,-0.39,63.15,62.52,88.0,-0.01,1.33,33.57,XR,NTT,BANKS
58,PGHH,13388.00,18062.58,-30.68,H-MC,5.73,200835.0,15.0,70111.0,-0.36,0.01,34.91,34.92,80.0,0.00,1.49,4.68,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.77,H-LC,2.94,147675.0,-2257.0,16850.0,-0.09,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.78,H-LC,3.58,214405.0,9905.0,31518.0,1.22,4.84,14.70,20.25,10.0,0.31,1.59,14.00,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.49,H-LC,3.71,187344.0,-12966.0,72952.0,0.45,-6.47,38.94,29.94,16.0,-0.18,1.39,29.43,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.26,H-LC,3.73,276120.0,10694.0,46084.0,-0.87,4.03,16.69,21.40,11.0,0.23,2.04,12.13,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-10.71,H-LC,5.22,174284.0,-27102.0,114714.0,-0.28,-13.46,65.82,43.51,27.0,-0.24,1.29,21.32,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,87955.0,-25594.0,65641.0,15.19,-22.54,74.63,35.27,268.0,-0.39,0.65,115.27,XR,NTT,HOTELS
31,HAPPSTMNDS,919.80,1488.71,-8.25,H-SC,8.85,64995.0,-31584.0,91318.0,4.60,-32.70,140.50,61.85,132.0,-0.35,0.48,14.60,AR,ATH,IT
66,SAMMAANCAP,170.35,326.00,-165.91,M-SC,7.48,75300.0,-26910.0,120299.0,3.33,-26.33,159.76,91.37,208.0,-0.22,0.56,22.24,XY25,NTT,FINANCE
81,TITAGARH,1097.23,1548.00,-8.89,H-SC,9.91,149336.0,-42679.0,121560.0,2.35,-22.23,81.40,41.08,158.0,-0.35,1.11,26.39,XY24,NTT,ENGINEERING
75,SURYODAY,189.78,240.00,44.74,H-SC,5.33,103490.0,-46436.0,86114.0,2.33,-30.97,83.21,26.46,135.0,-0.54,0.77,32.58,XR,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-4.69,M-SC,26.60,91725.0,-85090.0,165885.0,-2.69,-48.12,180.85,45.70,197.0,-0.51,0.68,6.70,XY24,NTT,TRAVEL
25,DABUR,505.20,735.00,-6.00,H-MC,3.82,204789.0,7761.0,81854.0,-1.85,3.94,39.97,45.49,102.0,0.09,1.52,17.49,XY24,BTT,FMCG
15,BLUESTARCO,1646.70,2326.38,9.78,H-MC,5.65,189550.0,24880.0,43085.0,-1.66,15.11,22.73,41.28,89.0,0.58,1.40,24.30,X40N,ATH,AC
14,BERGEPAINT,561.33,680.00,-19.46,H-MC,9.05,216675.0,-10664.0,58719.0,-1.56,-4.69,27.10,21.14,106.0,-0.18,1.60,22.13,XY24,NTT,PAINTS
87,VAIBHAVGBL,350.21,521.00,44.35,H-SC,10.21,96140.0,-55851.0,129972.0,-1.33,-36.75,135.19,48.77,125.0,-0.43,0.71,11.90,XR,NTT,JEWELLERY


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,99.57,M-SC,9.37,85995.0,-14782.0,14877.0,0.29,-14.67,17.30,0.09,245.0,-0.99,0.64,12.98,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,89.03,H-SC,16.24,119205.0,-23274.0,42795.0,1.08,-16.34,35.90,13.70,163.0,-0.54,0.88,40.69,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,117.08,M-SC,2.00,142364.0,-32195.0,32231.0,-1.13,-18.44,22.64,0.02,221.0,-1.00,1.05,23.45,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-18.14,H-SC,3.09,116910.0,-26893.0,53638.0,-0.08,-18.70,45.88,18.60,149.0,-0.50,0.87,29.01,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.34,H-SC,2.90,217683.0,-51984.0,88314.0,-0.96,-19.28,40.57,13.47,138.0,-0.59,1.61,10.10,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-26.44,H-LC,16.17,243025.0,-52909.0,146811.0,0.55,-17.88,60.41,31.73,1.0,-0.36,1.80,3.75,X40,BTT,IT
45,INFY,1461.46,2275.00,-18.16,H-LC,11.94,268339.0,6738.0,138892.0,0.19,2.58,51.76,55.67,3.0,0.05,1.99,8.84,X40,BTT,IT
47,ITC,411.90,452.00,-38.77,H-LC,2.94,147675.0,-2257.0,16850.0,-0.09,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.15,H-LC,9.28,278872.0,13832.0,82462.0,0.14,5.22,29.57,36.33,5.0,0.17,2.06,19.21,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-40.94,H-LC,11.93,200674.0,-15845.0,108745.0,0.47,-7.32,54.19,42.91,7.0,-0.15,1.49,5.09,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.80,H-MC,14.36,161124.0,34924.0,11246.0,0.80,27.67,6.98,36.59,108.0,3.11,1.19,43.65,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.35,H-LC,8.74,231388.0,46907.0,23463.0,-0.33,25.43,10.14,38.14,38.0,2.00,1.71,38.73,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,14.36,161124.0,34924.0,11246.0,0.80,27.67,6.98,36.59,108.0,3.11,1.19,43.65,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,10.56,H-SC,11.19,131872.0,6575.0,52432.0,0.34,5.25,39.76,47.09,141.0,0.13,0.98,65.05,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,9.78,H-MC,5.65,189550.0,24880.0,43085.0,-1.66,15.11,22.73,41.28,89.0,0.58,1.40,24.30,X40N,ATH,AC
90,VOLTAS,1278.28,1918.49,-2.76,H-MC,7.37,203940.0,12198.0,83840.0,-0.74,6.36,41.11,50.08,99.0,0.15,1.51,13.59,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,11.38,178794.0,27881.0,29429.0,-0.54,18.47,16.46,37.97,93.0,0.95,1.32,48.45,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,11.38,178794.0,27881.0,29429.0,-0.54,18.47,16.46,37.97,93.0,0.95,1.32,48.45,AR,ATH,AUTO
61,RAJOOENG,102.50,143.33,-28.19,H-SC,8.66,108750.0,6250.0,34582.0,0.82,6.10,31.80,39.83,114.0,0.18,0.80,7.67,AR,ATH,MISC
42,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.09,140954.0,17618.0,116146.0,1.37,14.28,82.40,108.45,119.0,0.15,1.04,39.83,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,10.56,H-SC,11.19,131872.0,6575.0,52432.0,0.34,5.25,39.76,47.09,141.0,0.13,0.98,65.05,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-41.30,M-SC,5.23,99180.0,7682.0,71072.0,1.87,8.40,71.66,86.07,223.0,0.11,0.73,43.47,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,7100.00,L-SC,7.09,79006.0,-14804.0,91418.0,-1.29,-15.78,115.71,81.67,269.0,-0.16,0.58,53.00,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,87955.0,-25594.0,65641.0,15.19,-22.54,74.63,35.27,268.0,-0.39,0.65,115.27,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-0.47,H-SC,15.72,92652.0,-8311.0,102844.0,0.66,-8.23,111.00,93.63,148.0,-0.08,0.69,38.00,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-17.34,H-SC,12.36,93900.0,-4080.0,25954.0,-0.33,-4.16,27.64,22.32,152.0,-0.16,0.69,36.19,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-9.59,H-LC,6.92,96883.0,-862.0,38259.0,0.96,-0.88,39.49,38.26,35.0,-0.02,0.72,26.86,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-39.64,L-SC,23.26,87955.0,-25594.0,65641.0,15.19,-22.54,74.63,35.27,268.0,-0.39,0.65,115.27,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-3.47,H-MC,11.38,178794.0,27881.0,29429.0,-0.54,18.47,16.46,37.97,93.0,0.95,1.32,48.45,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.35,H-LC,8.74,231388.0,46907.0,23463.0,-0.33,25.43,10.14,38.14,38.0,2.00,1.71,38.73,X40N,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.00,-12.98,H-LC,7.30,173866.0,17689.0,8398.0,0.25,11.33,4.83,16.71,22.0,2.11,1.29,30.74,X40N,BTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.71,H-LC,5.22,174284.0,-27102.0,114714.0,-0.28,-13.46,65.82,43.51,27.0,-0.24,1.29,21.32,X40,BTT,PAINTS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.77
1,25,41.36
2,50,70.95


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.57
LC    36.15
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.51
X40      14.37
X40N     13.84
XY25     11.83
XR        9.08
AR        8.47
OX40N     7.43
X5K       2.23
MH        1.73
X200      1.39
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.93
H-MC    23.38
H-SC    22.73
M-SC    13.24
M-LC     7.18
M-MC     2.59
L-SC     1.60
L-LC     1.04
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.56,1.16,29.10
IT,11.91,-16.09,74.08
FINANCE,8.09,-19.27,63.40
BANKS,7.59,-11.13,58.32
MISC,5.85,-16.06,82.59
PAINTS,5.55,-16.24,39.14
ELECTRICAL,4.73,-9.90,57.49
AUTO,4.56,-9.53,57.19
AC,3.64,1.54,31.81


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2785673.0,23
XR,1064770.0,13
AR,1062766.0,10
X40,796122.0,11
X40N,495936.0,11
OX40N,454651.0,10
XY25,448199.0,8
SR,191635.0,2
X5K,137343.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2704714.0,27
M-SC,1456283.0,17
H-MC,1219278.0,18
H-LC,1128554.0,19
M-LC,407844.0,5
M-MC,328380.0,2
L-SC,244586.0,3
L-MC,58712.0,1
L-LC,43642.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      916950.0      6
M-SC       XY24      858101.0      7
H-SC       AR        647590.0      5
           XR        512404.0      6
H-LC       X40       510254.0      6
H-MC       XY24      475424.0      5
M-MC       XY24      328380.0      2
H-LC       AR        228096.0      2
H-MC       X40       205162.0      4
H-SC       OX40N     197309.0      4
H-MC       XY25      195319.0      2
H-SC       SR        191635.0      2
H-LC       X40N      186662.0      5
H-SC       X40N      186394.0      3
M-LC       XY24      164602.0      2
M-SC       AR        157651.0      2
L-SC       XR        157059.0      2
M-LC       X5K       129514.0      1
H-MC       X40N      122880.0      3
M-SC       OX40N     121376.0      4
           XY25      120299.0      1
           XR        118150.0      2
H-MC       XR        113111.0      1
M-LC       XR        105334.0      1
L-SC       OX40N      87527.0      1
M-SC       X40        80706.0      1
H-LC       XY25       80545.0      3
           X200       72952.0      1
L-MC       XR         58712.0      1
H-SC       MH         52432.0      1
H-MC       OX40N      48439.0      1
L-LC       XY25       43642.0      1
H-LC       XY24       42216.0      1
H-MC       MH         29514.0      1
           AR         29429.0      1
M-LC       XY25        8394.0      1
H-LC       X5K         7829.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
